This is the notebook to train on the large dataset provided by Davut.  Specifically on the one with 32 datapoints.

In [60]:
%load_ext autoreload
%autoreload 2

# Define autroreload so that it doesn't cause pain in the ass when we change the functions and run this notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
# Import all the models, training functions, manipulators here

# Define the relative paths, append it to the system path
import sys
from pathlib import Path
project_root = Path.cwd().resolve().parents[3]
sys.path.append(str(project_root))

# Import shenanigans
from defs.helper_functions.training_functions import *
from defs.helper_functions.data_manipulation_functions import *
from defs.helper_functions.create_model_and_optimizer_functions import *

# Time, to precisely: time
import time

In [62]:
# Set device

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")  # Force CPU for testing purposes
print(f"Using device: {device}")

Using device: cuda


In [63]:
# Import and manipulate all the dataset required

df = pd.read_csv(rf"{project_root}\datasets\airfoil_dataset_32pts_200foils.csv") # read csv
df = datareform_tuple_coordinates(df=df, which_coord="y") # take in the y vals
shuffled_df = df.sample(frac=1, random_state=69).reset_index(drop=True) # shuffle the dataframe
input_tensor, target_tensor = seperate_transform(df=shuffled_df, target_coord=35) # get the target and input tensors
input_tensor = input_tensor.to(device)
target_tensor = target_tensor.to(device)
del df # Clear memory

In [64]:
print(input_tensor)
print(target_tensor)

tensor([[-2.2844e-03, -5.2456e-04,  5.2761e-03,  ...,  9.0000e+06,
          1.0000e-01,  8.0000e+00],
        [-2.4384e-03, -3.2231e-03, -5.5914e-03,  ...,  9.0000e+06,
          1.0000e-01,  2.0000e+01],
        [-6.1406e-03, -5.3207e-03, -3.4293e-03,  ...,  6.0000e+06,
          1.0000e-01,  0.0000e+00],
        ...,
        [-5.3469e-03, -2.5520e-03,  4.3910e-03,  ...,  9.0000e+06,
          1.0000e-01,  5.0000e+00],
        [-4.8743e-03, -6.5436e-03, -1.2151e-02,  ...,  9.0000e+06,
          1.0000e-01,  1.2000e+01],
        [-7.8454e-03, -4.6946e-03,  2.5393e-03,  ...,  3.0000e+06,
          1.0000e-01,  1.7000e+01]], device='cuda:0')
tensor([[ 0.1252,  1.4755, -0.2937],
        [ 0.0611,  1.8706, -0.0373],
        [ 0.0091,  0.3397, -0.0704],
        ...,
        [ 0.0108,  0.9608, -0.1011],
        [ 0.0119,  1.2228,  0.0336],
        [ 0.0999,  1.3706, -0.1453]], device='cuda:0')


In [65]:
"""
Trial 1, define configs for Trial 1 here, set device
"""

import torch.nn as nn

cfg_shape_net = {
    'input_dim': 32,
    'output_dim': 3,
    'hidden_units': [8,16,16,8],
    'shape_activation': nn.Tanh
}

cfg_param_net = {
    'input_dim': 3,
    'hidden_units': [8,16,16,8],
    'param_activation': nn.Tanh
}

NIF_model, NIF_optimizer = create_NIF_Pointwise_and_optimizer(cfg_shape_net=cfg_shape_net, cfg_param_net=cfg_param_net, lr=0.00005)
NIF_model = NIF_model.to(device)

In [66]:
# Show the model

from torchviz import make_dot

# Create and move inputs to device
show_tensor = torch.randn(5, cfg_shape_net['input_dim'] + cfg_param_net['input_dim']).to(device)

# Forward pass
output = NIF_model(show_tensor)

make_dot(output, params=dict(NIF_model.named_parameters())).render("model_graph", format="png")

'model_graph.png'

In [67]:
# Now that we have imported data, defined model, and seen what its like, we'll train here


def mse_loss(predictions, targets): return torch.mean((predictions - targets) ** 2)
torch.cuda.synchronize()
start = time.perf_counter()
epoch_train = train_one_epoch(model= NIF_model, optimizer= NIF_optimizer, criterion=mse_loss, inputs= input_tensor, targets= target_tensor, batch_size= 1, device= device, lambda_reg= 0)
torch.cuda.synchronize()
end = time.perf_counter()
print(f"Training took {end - start} seconds")

Rows 0 to 0, loss: 0.7981399893760681
Rows 1 to 1, loss: 1.2189537286758423
Rows 2 to 2, loss: 0.05383000522851944
Rows 3 to 3, loss: 0.41053757071495056
Rows 4 to 4, loss: 0.9220575094223022
Rows 5 to 5, loss: 0.004975292831659317
Rows 6 to 6, loss: 0.07838158309459686
Rows 7 to 7, loss: 0.42707571387290955
Rows 8 to 8, loss: 0.2098681628704071
Rows 9 to 9, loss: 1.3734462261199951
Rows 10 to 10, loss: 0.012034796178340912
Rows 11 to 11, loss: 0.011134921573102474
Rows 12 to 12, loss: 0.7179731726646423
Rows 13 to 13, loss: 0.2951849400997162
Rows 14 to 14, loss: 0.43129962682724
Rows 15 to 15, loss: 0.89748615026474
Rows 16 to 16, loss: 0.5960977077484131
Rows 17 to 17, loss: 0.826014518737793
Rows 18 to 18, loss: 0.40234825015068054
Rows 19 to 19, loss: 0.26593729853630066
Rows 20 to 20, loss: 0.8622636795043945
Rows 21 to 21, loss: 1.3215283155441284
Rows 22 to 22, loss: 0.5646133422851562
Rows 23 to 23, loss: 0.14482855796813965
Rows 24 to 24, loss: 0.005993376486003399
Rows 25 to

In [74]:
# Brief trial to ensure the model updates, we will calculate the MSE for the first row of data

start_idx = 4

trial_input = input_tensor[start_idx:start_idx+1].to(device)

result = NIF_model(trial_input)

trial_target = target_tensor[start_idx:start_idx+1].to(device)
trial_mse = mse_loss(result, trial_target).item()
print("Trial MSE for the first input-target pair:")

print(trial_mse)

Trial MSE for the first input-target pair:
0.1301993429660797
